# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 9:28AM,240036,12,HH-35244,Hush Hush,Released
1,Jun 21 2022 9:28AM,240035,12,HH-35246,Hush Hush,Released
2,Jun 21 2022 9:28AM,240035,12,HH-35248,Hush Hush,Released
3,Jun 21 2022 9:28AM,240035,12,HH-35253,Hush Hush,Released
4,Jun 21 2022 9:28AM,240035,12,HH-35254,Hush Hush,Released
5,Jun 21 2022 9:28AM,240035,12,HH-35242,Hush Hush,Released
6,Jun 21 2022 9:28AM,240035,12,HH-35255,Hush Hush,Released
7,Jun 21 2022 9:28AM,240035,12,HH-35256,Hush Hush,Released
8,Jun 21 2022 9:28AM,240035,12,HH-35264,Hush Hush,Released
9,Jun 21 2022 9:28AM,240035,12,HH-35265,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,240032,Released,38
49,240033,Released,39
50,240035,Released,13
51,240036,Released,1
52,240038,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240032,NaN,NaN,38.0
240033,NaN,NaN,39.0
240035,NaN,NaN,13.0
240036,NaN,NaN,1.0
240038,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239930,0.0,0.0,1.0
239934,0.0,0.0,1.0
239943,0.0,1.0,0.0
239949,0.0,0.0,28.0
239956,0.0,20.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239930,0,0,1
239934,0,0,1
239943,0,1,0
239949,0,0,28
239956,0,20,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239930,0,0,1
1,239934,0,0,1
2,239943,0,1,0
3,239949,0,0,28
4,239956,0,20,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239930,,,1
1,239934,,,1
2,239943,,1,
3,239949,,,28
4,239956,,20,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 9:28AM,240036,12,Hush Hush
1,Jun 21 2022 9:28AM,240035,12,Hush Hush
14,Jun 21 2022 9:27AM,240038,10,"Nextsource Biotechnology, LLC"
15,Jun 21 2022 9:26AM,240033,10,ISDIN Corporation
54,Jun 21 2022 9:26AM,240032,10,ISDIN Corporation
92,Jun 21 2022 9:10AM,240030,10,ISDIN Corporation
93,Jun 21 2022 8:59AM,240029,20,"Exact-Rx, Inc."
100,Jun 21 2022 8:57AM,240028,10,ISDIN Corporation
101,Jun 21 2022 8:56AM,240027,10,Emerginnova
105,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 9:28AM,240036,12,Hush Hush,,,1
1,Jun 21 2022 9:28AM,240035,12,Hush Hush,,,13
2,Jun 21 2022 9:27AM,240038,10,"Nextsource Biotechnology, LLC",,,1
3,Jun 21 2022 9:26AM,240033,10,ISDIN Corporation,,,39
4,Jun 21 2022 9:26AM,240032,10,ISDIN Corporation,,,38
5,Jun 21 2022 9:10AM,240030,10,ISDIN Corporation,,,1
6,Jun 21 2022 8:59AM,240029,20,"Exact-Rx, Inc.",,,7
7,Jun 21 2022 8:57AM,240028,10,ISDIN Corporation,,,1
8,Jun 21 2022 8:56AM,240027,10,Emerginnova,,,4
9,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 9:28AM,240036,12,Hush Hush,1,,
1,Jun 21 2022 9:28AM,240035,12,Hush Hush,13,,
2,Jun 21 2022 9:27AM,240038,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 21 2022 9:26AM,240033,10,ISDIN Corporation,39,,
4,Jun 21 2022 9:26AM,240032,10,ISDIN Corporation,38,,
5,Jun 21 2022 9:10AM,240030,10,ISDIN Corporation,1,,
6,Jun 21 2022 8:59AM,240029,20,"Exact-Rx, Inc.",7,,
7,Jun 21 2022 8:57AM,240028,10,ISDIN Corporation,1,,
8,Jun 21 2022 8:56AM,240027,10,Emerginnova,4,,
9,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 9:28AM,240036,12,Hush Hush,1,,
1,Jun 21 2022 9:28AM,240035,12,Hush Hush,13,,
2,Jun 21 2022 9:27AM,240038,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 21 2022 9:26AM,240033,10,ISDIN Corporation,39,,
4,Jun 21 2022 9:26AM,240032,10,ISDIN Corporation,38,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 9:28AM,240036,12,Hush Hush,1.0,NaN,NaN
1,Jun 21 2022 9:28AM,240035,12,Hush Hush,13.0,NaN,NaN
2,Jun 21 2022 9:27AM,240038,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Jun 21 2022 9:26AM,240033,10,ISDIN Corporation,39.0,NaN,NaN
4,Jun 21 2022 9:26AM,240032,10,ISDIN Corporation,38.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 9:28AM,240036,12,Hush Hush,1.0,0.0,0.0
1,Jun 21 2022 9:28AM,240035,12,Hush Hush,13.0,0.0,0.0
2,Jun 21 2022 9:27AM,240038,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Jun 21 2022 9:26AM,240033,10,ISDIN Corporation,39.0,0.0,0.0
4,Jun 21 2022 9:26AM,240032,10,ISDIN Corporation,38.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5760061,268.0,44.0,61.0
12,720050,15.0,0.0,0.0
18,240025,1.0,0.0,0.0
19,1200014,19.0,4.0,0.0
20,959952,10.0,43.0,2.0
21,719936,3.0,0.0,0.0
22,479937,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5760061,268.0,44.0,61.0
1,12,720050,15.0,0.0,0.0
2,18,240025,1.0,0.0,0.0
3,19,1200014,19.0,4.0,0.0
4,20,959952,10.0,43.0,2.0
5,21,719936,3.0,0.0,0.0
6,22,479937,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,268.0,44.0,61.0
1,12,15.0,0.0,0.0
2,18,1.0,0.0,0.0
3,19,19.0,4.0,0.0
4,20,10.0,43.0,2.0
5,21,3.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,268.0
1,12,Released,15.0
2,18,Released,1.0
3,19,Released,19.0
4,20,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,18,19,20,21,22
Status,,,,,,,
Completed,61.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,44.0,0.0,0.0,4.0,43.0,0.0,0.0
Released,268.0,15.0,1.0,19.0,10.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,18,19,20,21,22
0,Completed,61.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,44.0,0.0,0.0,4.0,43.0,0.0,0.0
2,Released,268.0,15.0,1.0,19.0,10.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,18,19,20,21,22
0,Completed,61.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,44.0,0.0,0.0,4.0,43.0,0.0,0.0
2,Released,268.0,15.0,1.0,19.0,10.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()